In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
import json
import numpy as np
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### never formatted Xartist

In [2]:
master_df = pd.read_csv('../components/classification/nodes.csv')

In [ ]:
master_df

In [3]:
y = master_df['label'].to_numpy()
np.save('../datasets/classification/numpy/y.npy', y)

In [4]:
A = np.load('../components/classification/A.npy')
np.save('../datasets/classification/numpy/A.npy', A)

In [5]:
master_df

,Unnamed: 0,id,name,followers,popularity,label
0,0,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72,2
1,1,5Matrg5du62bXwer29cU5T,Benny The Butcher,266547.0,64,3
2,2,4e0nWw2r4BoQSKPQ2zpU13,Ray Dalton,39139.0,71,5
3,3,2HmmA3GhP0xsbRUiDllJUn,Pep & Rash,77599.0,39,0
4,4,7jZMxhsB8djyIbYmoiJSTs,Kiana Ledé,1351452.0,64,4
...,...,...,...,...,...,...
1917,1917,2dHiQ5ggS4E4VguS5f9Ap4,Rimi Tomy,300705.0,45,1
1918,1918,3BeQqzKdlARoOd6y30kCO2,Pete Rock,298190.0,53,3
1919,1919,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,538974.0,61,0
1920,1920,6JtWu7EQUhFCLNZU92kq1o,Schoolboy,10153.0,25,0


In [18]:
mat = master_df.iloc[:, 3:5]
scaler = MinMaxScaler()
Xartist = scaler.fit_transform(mat)
np.save('../components/classification/Xartist.npy', Xartist)

### features

In [19]:
def get_client():
    load_dotenv(dotenv_path=".env")
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

    auth_manager = SpotifyClientCredentials(
        client_id=client_id, client_secret=client_secret, requests_timeout=5
    )
    sp = spotipy.Spotify(auth_manager=auth_manager)
    return sp
sp = get_client()

In [20]:
with open('../components/classification/id_to_ind.json', 'r') as f:
    artist_map = json.load(f)

In [21]:
artists = list(artist_map.keys())

In [22]:
artists

['4lDiJcOJ2GLCK6p9q5BgfK',
 '5Matrg5du62bXwer29cU5T',
 '4e0nWw2r4BoQSKPQ2zpU13',
 '2HmmA3GhP0xsbRUiDllJUn',
 '7jZMxhsB8djyIbYmoiJSTs',
 '4N874uPqBka1QiCvnCVOtr',
 '4kwEd1P9j15ZqUVP5zK7Pv',
 '23fqKkggKUBHNkbKtXEls4',
 '360IAlyVv4PCEVjgyMZrxK',
 '5ttgIeUVka6FLyi00Uu5h8',
 '2Z9KL8Zmqx5Sg3cd7Fldhl',
 '1XLjkBxFokuDTlHt0mQkRe',
 '4kYSro6naA4h99UJvo89HB',
 '5qZLYtTL3ldN6Uw7OXHESR',
 '2GHclqNVjqGuiE5mA7BEoc',
 '7H3TyvM3dpW78cru0Oaui9',
 '6hyMWrxGBsOx6sWcVj1DqP',
 '5uCXJWo3WoXgqv3T1RlAbh',
 '04FsFHujpuqonrPXZrEJlK',
 '1Sl3idO2XzH3AVb6nOjOxs',
 '1yzgJmZf5CxtFfGfRFm4Si',
 '6y5amJcTjeDgLXIjtQLMst',
 '78xUyw6FkVZrRAtziFdtdu',
 '1xr2G8Hlx4QWmT9HaUbmoO',
 '19MVxKZZdPj2X0F8pi0OCT',
 '0AsThoR4KZSVktALiNcQwW',
 '6vWDO969PvNqNYHIOW5v0m',
 '0huGjMyP507tBCARyzSkrv',
 '2Hjj68yyUPiC0HKEOigcEp',
 '5p7mWeX4RlvqBCxylfiXdE',
 '3Mcii5XWf6E0lrY3Uky4cA',
 '4xnihxcoXWK3UqryOSnbw5',
 '5wPoxI5si3eJsYYwyXV4Wi',
 '12SPNXi0aDpFt0rMVbmLrr',
 '0hCNtLu0JehylgoiP8L4Gh',
 '3gggmBN0erstm3YJvEGe3t',
 '6U3ybJ9UHNKEdsH7ktGBZ7',
 

In [27]:
# normalize: loudness, tempo, duration
# 4, 10, 11
# remove: type, id, uri, track_href, analysis_url
def get_audio_features_for_artists(artists):
    # map from id -> features
    id_to_features = defaultdict(dict)
    tracks = set()
    artists_to_ids = defaultdict(list)

    def get_top_track_ids(aid):
        nonlocal tracks, artists_to_ids
        resp = sp.artist_top_tracks(aid)
        ids = []
        for track in resp["tracks"]:
            id_ = track["id"]
            tracks.add(id_)
            artists_to_ids[aid].append(id_)
    
        return ids

    def get_audio_features(ids):
        nonlocal id_to_features
        resp = sp.audio_features(ids)
        for i in range(0, len(ids)):
            id_to_features[ids[i]] = resp[i]

    i = 0
    for a in artists:
        if i % 50 == 0: print("Artist #: ", i)
        get_top_track_ids(a)
        i += 1

    tracks = list(tracks)
    print(len(tracks)) # should be in the 29000 range
    chunk_size = 100
    for i in range(0, len(tracks), chunk_size):
        if (i % 1000) == 0: print("Feature analysis: ", i)
        track_chunk = tracks[i:i + chunk_size]
        get_audio_features(track_chunk)
    return id_to_features, artists_to_ids
id_to_features, artists_to_ids = get_audio_features_for_artists(artists)

Artist #:  0
Artist #:  50
Artist #:  100
Artist #:  150
Artist #:  200
Artist #:  250
Artist #:  300
Artist #:  350
Artist #:  400
Artist #:  450
Artist #:  500
Artist #:  550
Artist #:  600
Artist #:  650
Artist #:  700
Artist #:  750
Artist #:  800
Artist #:  850
Artist #:  900
Artist #:  950
Artist #:  1000
Artist #:  1050
Artist #:  1100
Artist #:  1150
Artist #:  1200
Artist #:  1250
Artist #:  1300
Artist #:  1350
Artist #:  1400
Artist #:  1450
Artist #:  1500
Artist #:  1550
Artist #:  1600
Artist #:  1650
Artist #:  1700
Artist #:  1750
Artist #:  1800
Artist #:  1850
Artist #:  1900
17802
Feature analysis:  0
Feature analysis:  1000
Feature analysis:  2000
Feature analysis:  3000
Feature analysis:  4000
Feature analysis:  5000
Feature analysis:  6000
Feature analysis:  7000
Feature analysis:  8000
Feature analysis:  9000
Feature analysis:  10000
Feature analysis:  11000
Feature analysis:  12000
Feature analysis:  13000
Feature analysis:  14000
Feature analysis:  15000
Featur

/Users/willcorcoran__/opt/anaconda3/envs/spotify/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/willcorcoran__/opt/anaconda3/envs/spotify/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


TypeError: can only concatenate list (not "float") to list

In [57]:
print(artists_to_ids)
def get_median_audio_features_for_artists(artists, artists_to_ids, id_to_features):
    artist_to_features = defaultdict(list)
    last = ''
    for a in artists:
        features = []
        for id_ in artists_to_ids[str(a)]:
            features_dic = id_to_features[id_]
            if features_dic is None: continue
            pop_keys = ["type", "id", "uri", "track_href", "analysis_url"]
            for p in pop_keys:
                if features_dic is None or p not in features_dic.keys(): continue
                features_dic.pop(p)
            features.append(list(features_dic.values()))
        if (a == '7F7vWRGRoB2xaJAtQ22TXV'):
            artist_to_features[a] = artist_to_features[last]
            continue
        arr = np.array(features)
        meds = np.median(arr, axis=0)
        artist_to_features[a] = meds.tolist()
        last = a

    rows = [[key] + value for key, value in artist_to_features.items()]
    df = pd.DataFrame(rows)
    df = df.iloc[:, 1:].reset_index(drop=True)
    scaling_columns = [4, 11, 12]
    scaler = MinMaxScaler()
    df[scaling_columns] = scaler.fit_transform(df[scaling_columns])
    features_mat = df.to_numpy()

    np.save("../components/classification/Xaudiofeatures.npy", features_mat)
    return features_mat

Xaudiofeatures = get_median_audio_features_for_artists(artists, artists_to_ids, id_to_features)

defaultdict(<class 'list'>, {'4lDiJcOJ2GLCK6p9q5BgfK': ['6LIgQ7TIoAx9nzpfvOq0Nq', '4xuVfUXkHe1YZZZWNICGcV', '552rCxBynmuPhmOLczMqCY', '01LYfVauZ0LlK5gok0HdTO', '3T33so3escAbwbrgcIqyRa', '7gK3S5NIkw1KdvdnUU1O5z', '7bdc7zbvOxYE6C85ix5FPp', '7lkYdyKCN44Lc5saKwIGhP', '3EcYkJfQKvKeaXrYzcWk6C', '7A9U3jZCSevE4n8qPAKtQK'], '5Matrg5du62bXwer29cU5T': ['59ClUavkkfRhryajZBn2Lf', '3Wp1UmoY6tylOCi7SIfdLA', '7d3mjNFdtUxeqc6zqYNnia', '1qzlwbV6SwHYOR3mhu6ZEd', '16uMpnprn7dUcSr64Y0CGY', '0FQeZmAKQLLnZC5iTN6JuR', '3ewqsanI2CJ3vwbP1fzQBE', '0SRRejumehaZSuf1BJ6rVA', '03jnWnj2qOrYofsyCTuHC6', '5ioDyG14CzzR5mIHA7UApb'], '4e0nWw2r4BoQSKPQ2zpU13': ['5YdnOm5990Kfq1Jodws98B', '5LWmVBXWFwspwJ77mecPVN', '1IeQnQxCs2YJN6u3wQbUyX', '2hBkoDhuGk2AtSCprEFDFP', '1xEIyhDMJA0vhJcPULuBFL', '6yHVyMdC08djbP86mdkP3G', '5nZQpofrOglw52mFVUE2BJ', '702sclnvy3bqY7KcUZarpr', '2gCZpfBPmMQ26t6yVPYMA1', '4qqaez1f2mrR4pixnmoNiY'], '2HmmA3GhP0xsbRUiDllJUn': ['7tu6IpmSey65T1TLcViLuQ', '15J54YXr8wiW7P6gj7g552', '0qF7Piw3F3mLJ1ni5Iw0bb', '6

In [58]:
Xaudiofeatures

array([[0.7005    , 0.6375    , 5.5       , ..., 0.50045058, 0.15508312,
        4.        ],
       [0.5465    , 0.7565    , 6.5       , ..., 0.09195445, 0.22199762,
        4.        ],
       [0.6585    , 0.7165    , 5.5       , ..., 0.4001328 , 0.14400595,
        4.        ],
       ...,
       [0.603     , 0.6845    , 5.5       , ..., 0.26278345, 0.60293177,
        4.        ],
       [0.565     , 0.909     , 4.5       , ..., 0.63124944, 0.45718987,
        4.        ],
       [0.632     , 0.767     , 6.5       , ..., 0.23427858, 0.50553447,
        4.        ]])

In [59]:
Xartist = np.load("../components/classification/Xartist.npy")
X_artist_2_cols = Xartist[:, :2]

In [60]:
Xartist

array([[2.40780258e-02, 7.26190476e-01],
       [3.20826162e-03, 6.30952381e-01],
       [4.69889972e-04, 7.14285714e-01],
       ...,
       [6.48873713e-03, 5.95238095e-01],
       [1.20850181e-04, 1.66666667e-01],
       [1.24083248e-02, 4.64285714e-01]])

In [62]:
Xfeatures = np.concatenate((X_artist_2_cols, Xaudiofeatures), axis=1)

In [63]:
np.save("../components/classification/Xfeatures.npy", Xfeatures)

In [65]:
np.save("../datasets/classification/numpy/X.npy", Xfeatures)